<h1 align="center"> Computation for Physicists </h1>
<h2 align="center"> <em> Solving Linear Equations</em> </h2>
<h2 align="center" > <a href="mailto:duan@unm.edu">Dr. Duan</a> (UNM) </h2>

# Error Analysis
- Total error = <span style='color:blue'> computational error </span> + <span style='color:red'> propagated data error </span>
$$\hat{f}(\hat{x}) - f(x) = \color{blue}{[\hat{f}(\hat{x}) - f(\hat{x})]} + \color{red}{[f(\hat{x}) - f(x)]},$$
where $f$ is the exact algorithm approximated by $\hat{f}$.
- Computational error = truncation error + rounding error.
    - Rounding error: due to the limited precision.
    - Truncation error: due to the algorithm.

In [ ]:
def mycos(x, tol=1e-6):
    # x = x % (2.0*pi) # move x to [0, 2*pi)
    res = 0. # result
    term = 1. # 1st term in the Taylor series
    n = 0 # power of x for the term
    while abs(term) >= tol: 
        res += term # add the term to the result
        term *= -x*x / ((n+1)*(n+2)) # compute the new term
        n += 2 # skip the odd power of x    
    # res += term # add the last term
    print(f"(-1)^{n//2} x^{n} / {n}! = {term}")
    return res

Is `tol` the tolerance of the absolute error or the relative error?

In [ ]:
import math
math.cos(3.) - mycos(3.) # dominated by truncation error

In [ ]:
math.cos(100.) - mycos(100.) # dominated by rounding error

- The _condition number_ 
$$\mathrm{cond} =  \left|\frac{[f(\hat{x}) - f(x)]/f(x)}{(\hat{x}-x)/x}\right| = \left|\frac{x f'(x)}{f(x)}\right|$$
measures the amplification of the data error in the original problem $f$.
- A problem is _ill-conditioned_ if $\mathrm{cond}\gg1$, which always amplifies the error no matter which algorithm you choose.

In [ ]:
math.tan(1.57078), math.tan(1.57079) # infinite cond at pi/2

- A reliable numerical solution is obtained for a <span style='color:red'>well-conditioned problem</span> (with a small propagated data error) using a <span style='color:blue'>stable algorithm</span> (with a small computational error).

# Homework 4
- The standard deviation of a dataset $\{x_1, x_2, \cdots, x_n\}$ is defined as
$$\sigma = \sqrt{\frac{1}{n}\sum_{i=1}^n (x_i - \bar{x})^2},$$
where $\bar{x}$ is the average value of $x_i$. It is straightforward to show that an equivalent definition of the standard deviation is
$$\sigma = \sqrt{\left(\frac{1}{n}\sum_{i=1}^n x_i^2\right)- \bar{x}^2}.$$
Which of the two algorithms is potentially faster to compute, and which is more accurate?

- Verify your conjecture with some example(s). You may find the NumPy functions `numpy.random.random()`, `numpy.sum()`, `numpy.mean()`, and `numpy.std()` useful.

In [ ]:
import numpy as np
from numpy.random import random

n = 1000 # number of values
data = np.empty((n,), dtype=np.float32)
data[:] = 10+random(n) # dataset
sigma = np.std(data)
mean = np.mean(data)
print(f"mean = {mean}, sigma = {sigma}")

In [ ]:
std1=np.sqrt(np.sum((data-mean)**2)/n)
std2=np.sqrt(np.sum(data**2)/n - mean**2)
print(f"std1 has relative error {(std1-sigma)/sigma}")
print(f"std2 has relative error {(std2-sigma)/sigma}")

In [ ]:
%timeit np.sqrt(np.sum((data-mean)**2)/n)

In [ ]:
%timeit np.sqrt(np.sum(data**2)/n - mean**2)

In [ ]:
%timeit np.std(data)

In [ ]:
import homework.hw4 as hw4

help(hw4)

In [ ]:
(hw4.std1(data)-sigma)/sigma, (hw4.std2(data)-sigma)/sigma

In [ ]:
%timeit hw4.std1(data)

In [ ]:
%timeit hw4.std2(data)

# Systems of Linear Equations
- Most of the numerical problems are reduced to solving a system of linear equations,
$$\mathbf{A} \boldsymbol{x} 
=\begin{bmatrix}
a_{00} & a_{01} & \cdots & a_{0,n_1} \\
a_{10} & a_{11} & \cdots & a_{1,n_1} \\
\vdots & \vdots & \ddots & \vdots \\
a_{m-1,0} & a_{m-1,1} & \cdots & a_{m-1,n-1}
\end{bmatrix}\begin{bmatrix}
x_0 \\ x_1 \\ \vdots \\ x_{n-1}
\end{bmatrix}
= \begin{bmatrix}
b_0 \\ b_1 \\ \vdots \\ b_{m-1}
\end{bmatrix}
= \boldsymbol{b}.$$
- For $\mathbf{A}\in\mathbb{R}^{m\times n}$, the solution exists if $\boldsymbol{b}\in\mathrm{span}(\mathbf{A})=\{\mathbf{A}\boldsymbol{x}: \boldsymbol{x}\in\mathbb{R}^n\}$.
- If $m=n$, the solution exists and is unique if $\mathbf{A}$ is _nonsingular_, which implies that $\mathbf{A}^{-1}$ exists, $\det(\mathbf{A})\neq0$, $\mathbf{A}$ has rank $n$, and $\mathbf{A}\boldsymbol{z}\neq\mathbf{0}$ for any nonzero vector $\boldsymbol{z}$.

- The [`linalg`](https://docs.scipy.org/doc/scipy/reference/tutorial/linalg.html) module in SciPy (and NumPy) provides the linear algebra tools for solving linear equations.

In [ ]:
import scipy.linalg as la
n = 100 # problem size
A = random(n*n).reshape((n, n)) # random matrix
x = random(n) # expected solution
b = A @ x # same as np.dot(A, x)

x1 = la.inv(A) @ b # x = A^-1 b
x2 = la.solve(A, b) # solve x from linear eqs
print('||x1-x|| = ', la.norm(x1-x)) # absolute error
print('||x2-x|| = ', la.norm(x2-x))

- Solving linear equations directly is faster than inverting the matrix.

In [ ]:
%timeit la.inv(A) @ b

In [ ]:
%timeit la.solve(A, b)

# Vector Norms
- The $p$-norm of a vector $\boldsymbol{x}$ is defined as 
$$\Vert\boldsymbol{x}\Vert_p = \left(\sum_{i=0}^{n-1} |x_i|^p \right)^{1/p}.$$
For example, $\Vert\boldsymbol{x}\Vert_1 = \sum_i |x_i|$, $\Vert\boldsymbol{x}\Vert_2 = \sqrt{\sum_i |x_i|^2}$, and $\Vert\boldsymbol{x}\Vert_\infty = \max |x_i|$.

![Unit Circles](fig/vecnorm.png)

In [ ]:
x = np.array([1, -2, 3])
print(f'Vector {x} has 2-norm {la.norm(x)}.')
print(f'It also has 1-norm {la.norm(x, 1)} and ∞-norm {la.norm(x, np.inf)}.')


- All norms are equivalent up to a constant factor.
- For any vector norm:
    - $\Vert\boldsymbol{x}\Vert >0 $ if $\boldsymbol{x}\neq\mathbf{0}$.
    - $\Vert \gamma \boldsymbol{x}\Vert = |\gamma|\cdot \Vert\boldsymbol{x}\Vert $ for any scalar $\gamma$.
    - $\Vert\boldsymbol{x} + \boldsymbol{y}\Vert \leq \Vert\boldsymbol{x}\Vert + \Vert\boldsymbol{y}\Vert $.

# Matrix Norms
- A vector-induced matrix norm is defined as the largest stretching it can do to a nonzero vector:
$$\Vert\mathbf{A}\Vert = \max_{\boldsymbol{x}\neq\mathbf{0}} 
\frac{\Vert \mathbf{A}\boldsymbol{x}\Vert}{\Vert\boldsymbol{x}\Vert}.$$
- It can be shown that
$$\Vert\mathbf{A}\Vert_1 = \max_j \sum_i |a_{ij}|
\qquad\mathrm{and}\qquad
\Vert\mathbf{A}\Vert_\infty = \max_i \sum_j |a_{i j}|.$$

In [ ]:
A = np.array([[1, 2], [3, 4]])
print(f"{A} has a Frobenius norm {la.norm(A)}.") # default is not 2-norm
print(f"It also has 1-norm {la.norm(A, 1)} and ∞-norm {la.norm(A, np.inf)}.")

- A vector-induced matrix norm has the following properties:
    - $\Vert\mathbf{A}\Vert > 0$ if $\mathbf{A}\neq\mathbf{0}$.
    - $\Vert\gamma\mathbf{A}\Vert = |\gamma|\cdot\Vert\mathbf{A}\Vert$ for any scalar $\gamma$.
    - $\Vert\mathbf{A} + \mathbf{B}\Vert\leq \Vert\mathbf{A}\Vert + \Vert\mathbf{B}\Vert$.
    - $\Vert\mathbf{A}\mathbf{B}\Vert \leq \Vert\mathbf{A}\Vert \cdot \Vert\mathbf{B}\Vert$.
    - $\Vert\mathbf{A} \boldsymbol{x}\Vert \leq \Vert\mathbf{A}\Vert \cdot \Vert\boldsymbol{x}\Vert$.

# Matrix Condition Number
- The condition number of a square nonsingular matrix $\mathbf{A}$ is defined by
$$\mathrm{cond}(\mathbf{A}) = \Vert\mathbf{A}\Vert \cdot \Vert\mathbf{A}^{-1}\Vert
= \left(\max_{\boldsymbol{x}\neq\mathbf{0}} \frac{\Vert\mathbf{A}\boldsymbol{x}\Vert}{\Vert\boldsymbol{x}\Vert}\right)
\left(\min_{\boldsymbol{x}\neq\mathbf{0}} \frac{\Vert\mathbf{A}\boldsymbol{x}\Vert}{\Vert\boldsymbol{x}\Vert}\right)^{-1}.$$
It is the ratio of the maximum stretching to the maximum shrinking a matrix does to any nonzero vector.
- $\mathrm{cond}(\mathbf{A})=\mathrm{cond}(\mathbf{A}^{-1})\geq 1$.
- $\mathrm{cond}(\mathbf{A})=\infty$ if $\mathbf{A}$ is singular.
- $\mathrm{cond}(\gamma \mathbf{A}) = \mathrm{cond}(\mathbf{A})$.
- $\mathrm{cond}(\mathbf{D})=(\max |d_{ii}|)/(\min |d_{ii}|)$ if $\mathbf{D}$ is diagonal.

- Suppose $\mathbf{A}\boldsymbol{x}=\boldsymbol{b}$ and $\mathbf{A}(\boldsymbol{x}+\Delta\boldsymbol{x})
=\boldsymbol{b} + \Delta\boldsymbol{b}$, then
$$\frac{\Vert\Delta\boldsymbol{x}\Vert}{\Vert\boldsymbol{x}\Vert}
\leq \mathrm{cond}(\mathbf{A})
\frac{\Vert\Delta\boldsymbol{b}\Vert}{\Vert\boldsymbol{b}\Vert}.$$
- An _ill-conditioned_ problem (with a very large condition number) can produce a poor solution because of the error amplification.

- Ill-conditioning can be because of the poor scaling.
- This can be because of the poor choice of units. Choose the right units so that the magnitudes of the matrix elements are comparable.

In [ ]:
from numpy.linalg import cond

A = np.array([[1, 1e-14], [2, -1e-14]])
x = np.array([1, 2e14])
b = A @ x

x1 = la.solve(A, b) # solver equilibrates A
print(f"cond(A) = {cond(A)}, x1 = {x1}.")

- Ill-conditioning may also come from a near singularity condition, e.g. a column or row vector is approximately a linear combination of other vectors.

In [ ]:
A = np.array([[1-1e-12, 1], [1, 1]])
x = np.array([1, 2])
b = A @ x

x1 = la.solve(A, b) 
cond(A), la.norm(x1-x)

# Gassian Elimination
- The basic technique of solving $\mathbf{A}\boldsymbol{x}=\boldsymbol{b}$ is called _Gaussian elimination_. In the first step, one uses a series of matrices $\mathbf{M}^{(k)}$ to make $\mathbf{A}$ upper triangular:
$$\mathbf{M}^{(n-2)}\cdots\mathbf{M}^{(0)} \mathbf{A}
=\mathbf{U}
=\begin{bmatrix} u_{00} & u_{01} & \cdots & u_{0,n-1} \\
0 & u_{11} & \cdots & u_{1,n-1} \\
\vdots & \vdots & \ddots & \vdots \\
0 & 0 & \cdots & u_{n-1,n-1}\end{bmatrix}.$$

- In the second step, one solves $\mathbf{U}\boldsymbol{x}=\boldsymbol{y} =\mathbf{M}^{(n-2)}\cdots\mathbf{M}^{(0)}\boldsymbol{b}$ with _backward substitution_:
$$x_i = \begin{cases}
y_{n-1}/u_{n-1,n-1} & \quad\mathrm{if}\, i=n-1, \\
\left(y_i - \sum_{j=i+1}^{n-1}u_{ij}x_j\right)u_{ii}^{-1} & \quad\mathrm{otherwise}.
\end{cases}$$

- Let $\mathbf{A}^{(k)}=\mathbf{M}^{(k-1)}\cdots\mathbf{M}^{(0)} \mathbf{A}$. Then
$\mathbf{M}^{(k)} = \mathbf{1}-\boldsymbol{m}^{(k)}\boldsymbol{e}_k^T$ annhilates the elements of the $k$th column vector of $\mathbf{A}^{(k)}$ below $a^{(k)}_{kk}$, where $a^{(k)}_{kk}$ is called the _pivot_, and
$$\boldsymbol{m}^{(k)} 
=\begin{bmatrix}
m^{(k)}_0 \\ \vdots \\ m^{(k)}_k \\ m^{(k)}_{k+1} \\ \vdots \\ m^{(k)}_{n-1}
\end{bmatrix}
= \begin{bmatrix}
0 \\ \vdots \\ 0  \\ a^{(k)}_{k+1,k}/a^{(k)}_{k,k} \\ \vdots \\ a^{(k)}_{n-1,k}/a^{(k)}_{k,k}
\end{bmatrix}.$$

- It can be shown that
\begin{align*}
\mathbf{L} &= (\mathbf{M}^{(k-1)}\cdots\mathbf{M}^{(0)})^{-1}
= \mathbf{1} + \sum_{k=0}^{n-2}\boldsymbol{m}^{(k)}\boldsymbol{e}_k^T\\
& = \begin{bmatrix}
1 & 0 & \cdots & 0 \\
l_{10} & 1 & \cdots &0 \\
\vdots & \vdots & \ddots & \vdots \\
l_{n-1,0} & l_{n-1,1} & \cdots & 1
\end{bmatrix}
\end{align*}
is lower triangular. Therefore, 
$$\mathbf{A}=\mathbf{L}\mathbf{U}.$$ 
This is known as the _LU factorization_.

- Explicit LU factorization is useful when many linear systems with the same $\mathbf{A}$ but different $\boldsymbol{b}$'s are to be solved.
- Given the LU factorization of $\mathbf{A}$, the first step of Gassian elimination can be achieved by solving $\mathbf{L}\boldsymbol{y}=\boldsymbol{b}$ with _forward substitution_:
$$y_i = \begin{cases}
b_0 & \quad\mathrm{if}\, i=0, \\
b_i - \sum_{j=0}^{i-1}l_{ij}y_j & \quad\mathrm{otherwise}.
\end{cases}$$

- Gassian elimination breaks down if the pivot $a^{(k)}_{kk}$ is 0. This can be fixed by permuting the rows of $\mathbf{A}^{(k)}$, which is known as (partial) pivoting.

- Gaussian elimination becomes more stable when the largest pivot is chosen at each step.

- $\mathbf{A}$ is singular if there is no nonzero pivot, but the LU factorization can still be completed.

In [ ]:
A = np.array([[2, 5, 8, 7], [5, 2, 2, 8], [7, 5, 6, 6], [5, 4, 4, 8]])
lu, piv = la.lu_factor(A) # LU factorization with pivoting
b1 = np.array([1, 1, 1, 1])
b2 = np.array([1, 2, 3, 4])
x1 = la.lu_solve((lu, piv), b1) # solve with LU and pivoting
x2 = la.lu_solve((lu, piv), b2)
la.norm(A @ x1 - b1), la.norm(A @ x2 - b2)

# Homework Problem
- Think how the LU factorization can be used to compute the determinent and inverse of a nonsingular matrix.
- Implement a simple function to compute the inverse of an arbitrary square matrix using  `lu_factor()` and `lu_solve()` in `scipy.linalg`. Document and validate the function.
- Hint: For $\mathbf{A}\mathbf{X}=\mathbf{1}$, you can treat $\mathbf{X}$ and $\mathbf{1}$ as collections of column vectors. `numpy.eye(n)` returns a $n\times n$ identity matrix.